# Chargement du fichier JSON dans un DataFrame Spark

In [51]:
%%pyspark
df_raw = spark.read.load('abfss://synapsedatalake5cloud@datalake5cloud.dfs.core.windows.net/data_601c8ae0-679c-4371-a2bb-bf0617596956_5078817c-92be-4caa-a136-39c8335ada2c.json', format='json')
display(df_raw.limit(10))

StatementMeta(pool, 4, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5fbfe36d-eaea-4b22-ad55-aeebefe8e23a)

# 2. Extraction et Flattening des Données: 

Le JSON contient une structure imbriquée, donc nous devons extraire les données de l'objet data.

In [52]:
# Extraction des informations de "data"
df = df_raw.selectExpr("explode(data) as data").select("data.*")


StatementMeta(pool, 4, 14, Finished, Available, Finished)

# 3. Nettoyage et Transformation des Données

- Sélection des colonnes pertinentes
- Conversion du updateTime (en millisecondes depuis l'époque Unix) en date
- Gestion des valeurs nulles

In [53]:
from pyspark.sql.functions import col, from_unixtime,when

# Sélection des colonnes et nettoyage des valeurs nulles
df_cleaned = df.select(
    col("encryptedUid").alias("user_id"),
    col("nickName").alias("nickname"),
    col("userPhotoUrl").alias("photo_url"),
    col("rank"),
    col("followerCount").alias("follower_count"),
    col("leaderboardUrl").alias("leaderboard_url"),
    from_unixtime(col("updateTime") / 1000).cast("timestamp").alias("update_time")
).fillna({"nickname": "Unknown", "photo_url": "", "follower_count": 0})


# Affichage des premières lignes après nettoyage
df_cleaned.show(5)


StatementMeta(pool, 4, 15, Finished, Available, Finished)

+--------------------+--------------------+---------+----+--------------+--------------------+-------------------+
|             user_id|            nickname|photo_url|rank|follower_count|     leaderboard_url|        update_time|
+--------------------+--------------------+---------+----+--------------+--------------------+-------------------+
|E4C2BCB6FDF2A2A7A...|     Ethcoin Founder|         |  29|          6174|https://www.binan...|2024-11-12 00:00:00|
|26D230E2086395890...|Anonymous User-ea...|         | 144|           900|https://www.binan...|2024-11-12 00:00:00|
|8FEB3EA2D767A2732...|            Dumbass1|         | 183|          2158|https://www.binan...|2024-11-12 00:00:00|
|61FB1D1ADC31E34B3...|        musthaveluck|         | 231|           489|https://www.binan...|2024-11-12 00:00:00|
|988EE5573E692A34D...|        Miracle55555|         | 254|             0|https://www.binan...|2024-11-12 00:00:00|
+--------------------+--------------------+---------+----+--------------+-------

# 4. Ajout de Colonnes Dérivées
nous  ajoutons des colonnes dérivées pour faciliter la visualisation, comme par exemple une colonne rank_category pour catégoriser les utilisateurs en fonction de leur rank.

In [54]:
# Ajout d'une colonne de catégorie de rang (par exemple, Top 50, 100, 500, etc.)
df_transformed = df_cleaned.withColumn(
    "rank_category",
    when(col("rank") <= 50, "Top 50")
    .when(col("rank") <= 100, "Top 100")
    .when(col("rank") <= 500, "Top 500")
    .otherwise("500+")
)

# Affichage des premières lignes après nettoyage
df_transformed.show(5)



StatementMeta(pool, 4, 16, Finished, Available, Finished)

+--------------------+--------------------+---------+----+--------------+--------------------+-------------------+-------------+
|             user_id|            nickname|photo_url|rank|follower_count|     leaderboard_url|        update_time|rank_category|
+--------------------+--------------------+---------+----+--------------+--------------------+-------------------+-------------+
|E4C2BCB6FDF2A2A7A...|     Ethcoin Founder|         |  29|          6174|https://www.binan...|2024-11-12 00:00:00|       Top 50|
|26D230E2086395890...|Anonymous User-ea...|         | 144|           900|https://www.binan...|2024-11-12 00:00:00|      Top 500|
|8FEB3EA2D767A2732...|            Dumbass1|         | 183|          2158|https://www.binan...|2024-11-12 00:00:00|      Top 500|
|61FB1D1ADC31E34B3...|        musthaveluck|         | 231|           489|https://www.binan...|2024-11-12 00:00:00|      Top 500|
|988EE5573E692A34D...|        Miracle55555|         | 254|             0|https://www.binan...|202

# 5. Écriture des Données dans Azure Synapse pour la Visualisation
Après le nettoyage et les transformations, nous pouvons écrire les données nettoyées dans un fichier dans le Data Lake pour la visualisation.

In [18]:
# Écriture des données transformées dans un fichier CSV
df_transformed.write.mode("overwrite").option("header", "true").csv("abfss://synapsedatalake5cloud@datalake5cloud.dfs.core.windows.net/cleaned_finance_data")


StatementMeta(pool, 2, 6, Finished, Available, Finished)

# 6. Visualisation des données